### Install libraries

In [1]:
!pip install pymupdf sentence-transformers numpy chromadb faiss-cpu qdrant-client weaviate-client pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.7/603.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220

### Loading + Chunking Docs

In [2]:
import fitz

In [3]:
def load_pdf(path):
  doc = fitz.open(path)
  text = ""
  for page in doc:
    text += page.get_text()
  return text

In [4]:
def chunk_text(text, chunk_size = 500, overlap = 100):
  chunks = []
  start = 0
  while start < len(text):
    chunks.append(text[start:start + chunk_size])
    start += chunk_size - overlap
  return chunks

In [5]:
pdf_text = load_pdf("/content/drive/MyDrive/Attention_Is_All_You_Need.pdf")

In [6]:
chunks = chunk_text(pdf_text)

In [7]:
print(f"Total chunks: {len(chunks)}")

Total chunks: 99


### Create Embeddings

In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer

In [9]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
embeddings = model.encode(chunks).astype("float32")

In [11]:
query = "What is multi head attention"
query_embedding = model.encode([query]).astype("float32")

## ChromaDB

In [12]:
import chromadb
from chromadb.config import Settings

In [15]:
client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = client.get_or_create_collection(name="pdf")

In [16]:
collection.add(
    documents=chunks,
    embeddings=embeddings.tolist(),
    ids=[str(i) for i in range(len(chunks))]
)

In [17]:
res = collection.query(query_texts=[query], n_results=3)
print("ChromaDB:", res["documents"][0])

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 35.4MiB/s]


ChromaDB: [' model to jointly attend to information from different representation\nsubspaces at different positions. With a single attention head, averaging inhibits this.\nMultiHead(Q, K, V ) = Concat(head1, ..., headh)W O\nwhere headi = Attention(QW Q\ni , KW K\ni , V W V\ni )\nWhere the projections are parameter matrices W Q\ni\n∈Rdmodel×dk, W K\ni\n∈Rdmodel×dk, W V\ni\n∈Rdmodel×dv\nand W O ∈Rhdv×dmodel.\nIn this work we employ h = 8 parallel attention layers, or heads. For each of these we use\ndk = dv = dmodel/h = 64.', 'Multi-Head Attention\nInstead of performing a single attention function with dmodel-dimensional keys, values and queries,\nwe found it beneficial to linearly project the queries, keys and values h times with different, learned\nlinear projections to dk, dk and dv dimensions, respectively. On each of these projected versions of\nqueries, keys and values we then perform the attention function in parallel, yielding dv-dimensional\n4To illustrate why the dot products 

## FAISS

In [18]:
import faiss

In [19]:
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

In [20]:
D, I = index.search(query_embedding, k = 3)
print("FAISS: ", [chunks[i] for i in I[0]])

FAISS:  [' model to jointly attend to information from different representation\nsubspaces at different positions. With a single attention head, averaging inhibits this.\nMultiHead(Q, K, V ) = Concat(head1, ..., headh)W O\nwhere headi = Attention(QW Q\ni , KW K\ni , V W V\ni )\nWhere the projections are parameter matrices W Q\ni\n∈Rdmodel×dk, W K\ni\n∈Rdmodel×dk, W V\ni\n∈Rdmodel×dv\nand W O ∈Rhdv×dmodel.\nIn this work we employ h = 8 parallel attention layers, or heads. For each of these we use\ndk = dv = dmodel/h = 64.', 'Multi-Head Attention\nInstead of performing a single attention function with dmodel-dimensional keys, values and queries,\nwe found it beneficial to linearly project the queries, keys and values h times with different, learned\nlinear projections to dk, dk and dv dimensions, respectively. On each of these projected versions of\nqueries, keys and values we then perform the attention function in parallel, yielding dv-dimensional\n4To illustrate why the dot products ge

## Qdrant

In [21]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

In [22]:
client = QdrantClient(":memory:")

In [24]:
collection_name = "pdf"
vector_size = embeddings.shape[1]

In [25]:
if client.collection_exists(collection_name):
  client.delete_collection(collection_name)

In [27]:
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=vector_size,
        distance=Distance.COSINE
      )
)

True

In [28]:
client.upsert(
    collection_name = "pdf",
    points = [
        PointStruct(id = i, vector = embeddings[i], payload = {"text": chunks[i]}) for i in range(len(chunks))
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [42]:
res = client.query_points(
    collection_name = "pdf",
    query = query_embedding[0].tolist(),
    limit = 3,
    with_payload = True
)

In [47]:
points = res.points

In [48]:
print("Qdrant:", [p.payload["text"] for p in points])

Qdrant: [' model to jointly attend to information from different representation\nsubspaces at different positions. With a single attention head, averaging inhibits this.\nMultiHead(Q, K, V ) = Concat(head1, ..., headh)W O\nwhere headi = Attention(QW Q\ni , KW K\ni , V W V\ni )\nWhere the projections are parameter matrices W Q\ni\n∈Rdmodel×dk, W K\ni\n∈Rdmodel×dk, W V\ni\n∈Rdmodel×dv\nand W O ∈Rhdv×dmodel.\nIn this work we employ h = 8 parallel attention layers, or heads. For each of these we use\ndk = dv = dmodel/h = 64.', 'Multi-Head Attention\nInstead of performing a single attention function with dmodel-dimensional keys, values and queries,\nwe found it beneficial to linearly project the queries, keys and values h times with different, learned\nlinear projections to dk, dk and dv dimensions, respectively. On each of these projected versions of\nqueries, keys and values we then perform the attention function in parallel, yielding dv-dimensional\n4To illustrate why the dot products ge

## Weaviate

In [74]:
import weaviate
from weaviate.connect import ConnectionParams
from weaviate.classes.config import Configure, Property, DataType
from weaviate.classes.init import Auth

In [64]:
WEAVIATE_URL="h7ypzvzoqw8plo7n5iawq.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY="bUYyR3hQZk85QnNlazZnM180dzEzN0FxeHpBV1paRUFXeU1ZaVo5NFdQYkV4NkVWSHNnZ2dwdFRNUS9vPV92MjAw"

In [67]:
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

In [68]:
print(client.is_ready())

True


In [69]:
client.connect()

In [70]:
COLLECTION = "PdfChunk"

In [72]:
if client.collections.exists(COLLECTION):
  client.collections.delete(COLLECTION)

In [75]:
client.collections.create(
    name=COLLECTION,
    properties=[
        Property(name="text", data_type=DataType.TEXT),
    ],
    vectorizer_config = Configure.Vectorizer.none()
)

In [76]:
collection = client.collections.get(COLLECTION)

for i, emb in enumerate(embeddings):
  collection.data.insert(
      properties={"text":chunks[i]},
      vector=emb.tolist(),
  )

In [77]:
res = collection.query.near_vector(
    near_vector=query_embedding[0].tolist(),
    limit=3
)

In [78]:
print("Weaviate:", [obj.properties["text"] for obj in res.objects])

Weaviate: [' model to jointly attend to information from different representation\nsubspaces at different positions. With a single attention head, averaging inhibits this.\nMultiHead(Q, K, V ) = Concat(head1, ..., headh)W O\nwhere headi = Attention(QW Q\ni , KW K\ni , V W V\ni )\nWhere the projections are parameter matrices W Q\ni\n∈Rdmodel×dk, W K\ni\n∈Rdmodel×dk, W V\ni\n∈Rdmodel×dv\nand W O ∈Rhdv×dmodel.\nIn this work we employ h = 8 parallel attention layers, or heads. For each of these we use\ndk = dv = dmodel/h = 64.', 'Multi-Head Attention\nInstead of performing a single attention function with dmodel-dimensional keys, values and queries,\nwe found it beneficial to linearly project the queries, keys and values h times with different, learned\nlinear projections to dk, dk and dv dimensions, respectively. On each of these projected versions of\nqueries, keys and values we then perform the attention function in parallel, yielding dv-dimensional\n4To illustrate why the dot products 

## Pinecone

In [82]:
from pinecone import Pinecone

In [83]:
PINECONE_API_KEY = "pcsk_LKvLu_Uu4q74dnJur4nkmC5zCwxJwrdAG7ThZ8YwgmxSAWmGBCGEDvSd6SgCZs58ndhbX"

In [87]:
pc = Pinecone(api_key = PINECONE_API_KEY)
index = pc.Index("pdf-index")

In [88]:
index.upsert(
    vectors = [(str(i), embeddings[i].tolist()) for i in range(len(chunks))]
)

UpsertResponse(upserted_count=99, _response_info={'raw_headers': {'date': 'Sun, 08 Feb 2026 14:08:03 GMT', 'content-type': 'application/json', 'content-length': '20', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '1', 'x-pinecone-request-logical-size': '152252', 'x-pinecone-request-latency-ms': '483', 'x-envoy-upstream-service-time': '220', 'x-pinecone-response-duration-ms': '485', 'grpc-status': '0', 'server': 'envoy'}})

In [89]:
res = index.query(
    vector = query_embedding[0].tolist(),
    top_k = 3,
    include_metadata =False
)

In [90]:
print("Pinecone:", [m["id"] for m in res["matches"]])

Pinecone: ['29', '27', '98']
